In [54]:
import requests
import asyncio

import json

# import nest_asyncio
# nest_asyncio.apply()

import numpy as np
import pandas as pd

import urllib.parse

In [55]:
toxic = pd.read_csv('data/raw/train_toxic.csv')

In [64]:
text = toxic[toxic['toxicity'] > 2].iloc[46].comment_text.replace('  ', ' ')

text

'i have a dick its bigger than yours hahaha'

In [4]:
urllib.parse.quote_plus('Fuck mama')

'Fuck+mama'

In [5]:
#!pip install watson-developer-cloud
!pip install --upgrade "ibm-watson>=5.2.3"

Requirement already up-to-date: ibm-watson>=5.2.3 in /usr/local/lib/python3.7/site-packages (5.3.0)
     |████████████████████████████████| 138 kB 3.6 MB/s eta 0:00:01
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.9
    Uninstalling urllib3-1.25.9:
      Successfully uninstalled urllib3-1.25.9
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

requests 2.24.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you'll have urllib3 1.26.7 which is incompatible.
ibm-cloud-sdk-core 3.13.2 requires requests<3.0.0,>=2.26.0, but you'll have requests 2.24.0 which is incompatible.
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [6]:
API_KEY = 'i_xtSOVEL1WLC-wIgo5HAjGaOqcPwkwv5peRSKR1gX1T'
API_URL = 'https://api.eu-gb.natural-language-understanding.watson.cloud.ibm.com/instances/b69e6f84-8f95-4f86-985b-294c699a1025/v1/analyze'

In [59]:
%%time

import json
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson.natural_language_understanding_v1 import Features, EntitiesOptions, KeywordsOptions, EmotionOptions, SentimentOptions

authenticator = IAMAuthenticator(API_KEY)
natural_language_understanding = NaturalLanguageUnderstandingV1(
    version='2021-03-25',
    authenticator=authenticator
)
natural_language_understanding.set_service_url(API_URL)

response = natural_language_understanding.analyze(
    text=text,
    features=Features(
        emotion=EmotionOptions(),
        sentiment=SentimentOptions()
#         keywords=KeywordsOptions(
#             emotion=True,
#             sentiment=True,
#             limit=2
#         )
    )
)

response = response.get_result()

print(json.dumps(response, indent=2))

{
  "usage": {
    "text_units": 1,
    "text_characters": 159,
    "features": 2
  },
  "sentiment": {
    "document": {
      "score": -0.997672,
      "label": "negative"
    }
  },
  "language": "en",
  "emotion": {
    "document": {
      "emotion": {
        "sadness": 0.473614,
        "joy": 0.006004,
        "fear": 0.091082,
        "disgust": 0.523435,
        "anger": 0.282748
      }
    }
  }
}
CPU times: user 41.7 ms, sys: 0 ns, total: 41.7 ms
Wall time: 1.03 s


In [78]:
!curl -u "apikey:i_xtSOVEL1WLC-wIgo5HAjGaOqcPwkwv5peRSKR1gX1T" "https://api.eu-gb.natural-language-understanding.watson.cloud.ibm.com/instances/b69e6f84-8f95-4f86-985b-294c699a1025/v1/analyze?version=2021-11-01&text=Fuck+mama+baby+sucker&features=keywords,entities&entities.emotion=true&entities.sentiment=true&keywords.emotion=true&keywords.sentiment=true"

{
  "usage": {
    "text_units": 1,
    "text_characters": 21,
    "features": 2
  },
  "language": "en",
  "keywords": [
    {
      "text": "Fuck mama baby sucker",
      "sentiment": {
        "score": -0.992999,
        "label": "negative"
      },
      "relevance": 0.5,
      "emotion": {
        "sadness": 0.084355,
        "joy": 0.190842,
        "fear": 0.008647,
        "disgust": 0.104548,
        "anger": 0.767937
      },
      "count": 1
    }
  ],
  "entities": []
}

In [3]:
result = {
    "usage": {
        "text_units": 1,
        "text_characters": 12,
        "features": 2
    },
    "sentiment": {
        "document": {
            "score": -0.99235,
            "label": "negative"
        }
    },
    "language": "en",
    "emotion": {
        "document": {
            "emotion": {
                "sadness": 0.110137,
                "joy": 0.071086,
                "fear": 0.011037,
                "disgust": 0.04831,
                "anger": 0.887245
            }
        }
    }
}

In [49]:
!pip install aiohttp

     |████████████████████████████████| 1.1 MB 2.6 MB/s eta 0:00:01
     |████████████████████████████████| 192 kB 9.7 MB/s eta 0:00:01
     |████████████████████████████████| 271 kB 12.7 MB/s eta 0:00:01
     |████████████████████████████████| 160 kB 8.6 MB/s eta 0:00:01     |██████████████████▍             | 92 kB 8.6 MB/s eta 0:00:01
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [7]:
import aiohttp
from aiohttp import BasicAuth

In [65]:
async def make_request(url, auth, headers, params):
    async with aiohttp.ClientSession(headers=headers) as session:
        async with session.get(url=url, params=params, auth=auth) as response:
            response = await response.read()
            response = response.decode('utf-8')
            response = json.loads(response)
            return response

In [9]:
import time

In [11]:
ts = time.time()
text = toxic[toxic['toxicity'] > 4].iloc[100].comment_text

print(text, '')

auth = BasicAuth('apikey', API_KEY)

params = {
    'version': '2021-11-01',
    'text': text,
    'features': 'emotion,sentiment',
    'emotion.document': 'true',
    'sentiment.document': 'true'
}

result = await make_request(API_URL, auth, params)

print(result)

sentiment_score = result['sentiment']['document']['score']
emotions_scores = result['emotion']['document']['emotion']

sentiment_index = 1
if sentiment_score < 0:
    sentiment_index = 0.5*(1 + sentiment_score) # weight = 5

sadness_index = 1 - emotions_scores['sadness'] # weight = 0.5
joy_index = emotions_scores['joy'] # weight = 3
fear_index = 1 - emotions_scores['fear'] # weight = 0.5
disgust_index = 1 - emotions_scores['disgust'] # weight = 0.5
anger_index = 1 - emotions_scores['anger'] # weight 2

indexes = [
    sentiment_index,
    sadness_index,
    joy_index,
    fear_index,
    disgust_index,
    anger_index
]

weights = [3, 1, 3, 1, 1, 3]

message_index = np.average(indexes, weights=weights)

print(message_index)

print(time.time() - ts)

can i eye fuck you you negro see grow mar utc  
{'usage': {'text_units': 1, 'text_characters': 46, 'features': 2}, 'sentiment': {'document': {'score': -0.717678, 'label': 'negative'}}, 'language': 'en', 'emotion': {'document': {'emotion': {'sadness': 0.147835, 'joy': 0.120727, 'fear': 0.14943, 'disgust': 0.316039, 'anger': 0.480694}}}}
0.39418983333333335
1.2197554111480713


In [12]:
!pip install torch==1.10.0+cpu torchvision==0.11.1+cpu torchaudio==0.10.0+cpu -f https://download.pytorch.org/whl/cpu/torch_stable.html

Looking in links: https://download.pytorch.org/whl/cpu/torch_stable.html
     |████████████████████████████████| 199.3 MB 174 kB/s eta 0:00:011    |██████                          | 36.9 MB 11.7 MB/s eta 0:00:14     |███████▏                        | 44.6 MB 11.8 MB/s eta 0:00:14     |██████████▏                     | 63.5 MB 1.9 MB/s eta 0:01:11     |██████████████                  | 87.4 MB 3.4 MB/s eta 0:00:34     |██████████████▎                 | 88.7 MB 3.4 MB/s eta 0:00:33     |████████████████████████▎       | 151.5 MB 12.0 MB/s eta 0:00:04     |██████████████████████████      | 161.5 MB 21.3 MB/s eta 0:00:02
     |████████████████████████████████| 16.2 MB 19.6 MB/s eta 0:00:01   |████████████████▋               | 8.4 MB 382 kB/s eta 0:00:21
     |████████████████████████████████| 2.7 MB 4.5 MB/s eta 0:00:01     |███                             | 256 kB 4.5 MB/s eta 0:00:01
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [13]:
!pip install transformers

     |████████████████████████████████| 127 kB 3.7 MB/s eta 0:00:01
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.7
    Uninstalling urllib3-1.26.7:
      Successfully uninstalled urllib3-1.26.7
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

ibm-cloud-sdk-core 3.13.2 requires requests<3.0.0,>=2.26.0, but you'll have requests 2.24.0 which is incompatible.
ibm-cloud-sdk-core 3.13.2 requires urllib3<2.0.0,>=1.26.0, but you'll have urllib3 1.25.11 which is incompatible.
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [1]:
from transformers import pipeline

nlp = pipeline("fill-mask")
print(nlp(f"HuggingFace is creating a {nlp.tokenizer.mask_token} that the community uses to solve NLP tasks."))

No model was supplied, defaulted to distilroberta-base (https://huggingface.co/distilroberta-base)


RuntimeError: [enforce fail at CPUAllocator.cpp:68] . DefaultCPUAllocator: can't allocate memory: you tried to allocate 154414080 bytes. Error code 12 (Cannot allocate memory)

In [108]:
text

'wtf  i haven t even edited anything within the past day  after the others told me to stop '

In [2]:
!pip install fitbert

You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [20]:
bad_words = pd.read_csv('data/raw/bad-words.csv', names=['words'])
bad_words = bad_words['words'].to_list()

bad_words

['jigaboo',
 'mound of venus',
 'asslover',
 's&m',
 'queaf',
 'whitetrash',
 'meatrack',
 'ra8s',
 'pimp',
 'urine',
 'whit',
 'randy',
 'herpes',
 'niglet',
 'narcotic',
 'pudboy',
 'rimming',
 'boner',
 'pornography',
 'poop chute',
 'israel',
 'dong',
 'slanteye',
 'muffdiving',
 'jiggabo',
 'assassination',
 'peepshpw',
 'popimp',
 'girl on',
 'testicles',
 'laid',
 'molestor',
 'peni5',
 'tranny',
 'barface',
 'hell',
 'arsehole',
 'pissed',
 'sixsixsix',
 'execute',
 'shitty ',
 'conspiracy',
 'hamas',
 'cunilingus',
 'bitcher',
 'muslim',
 'pee',
 'niggled',
 'muffindiver',
 'cocksman',
 'scag',
 'aroused',
 'niggling',
 'fingerfucker ',
 'nlggor',
 'niggaz',
 'assfuck',
 'slant',
 'urinate',
 'mothafucked ',
 'fungus',
 'retard',
 'gummer',
 'venus mound',
 'alla',
 'spaghettinigger',
 'piss',
 'mickeyfinn',
 'fuckers',
 'jizzim',
 'tramp',
 'quashie',
 'prince albert piercing',
 'hardon',
 'menage a trois',
 'bukkake',
 'shag',
 'australian',
 'raped',
 'buggery',
 'deth',
 '

In [24]:
import re

regexp = (r'\b(%s)\b' % '|'.join(bad_words))

r = re.compile(regexp, re.IGNORECASE)

In [48]:
message = 'fuck you bitch baby, hello world, fuck!'

In [49]:
for match in re.finditer(r, message):
    s1, s2 = match.span()
    message[0:34] +  + message[38:]

(0, 4)
(9, 14)
(34, 38)


In [ ]:
sorted(x.items(), key=lambda item: item[1])